In [2]:
import backtrader as bt
import pandas as pd
from datetime import datetime

In [14]:
class MultiStrategy(bt.Strategy):
    params = (
        ('sma_fast', 10),
        ('sma_slow', 30),
        ('rsi_period', 14),
        ('macd_fast', 12),
        ('macd_slow', 26),
        ('macd_signal', 9),
        ('bollinger_period', 20),
        ('devfactor', 2),
        ('atr_period', 14),
        ('adx_period', 14),
        ('stoploss', 0.02),
        ('takeprofit', 0.04),
        ('risk_per_trade', 0.02),
        ('vwap_period', 20),
        ('obv_ema', 21)
    )

    def __init__(self):
        self.orders = {}  # 多股票订单管理
        self.indicators = {}  # 多股票指标存储
        
        for d in self.datas:
            # 基础指标
            self.indicators[d] = {
                'sma_fast': bt.indicators.SMA(d.close, period=self.p.sma_fast),
                'sma_slow': bt.indicators.SMA(d.close, period=self.p.sma_slow),
                'rsi': bt.indicators.RSI(d.close, period=self.p.rsi_period),
                'macd': bt.indicators.MACD(d,
                    period_me1=self.p.macd_fast,
                    period_me2=self.p.macd_slow,
                    period_signal=self.p.macd_signal),
                'bollinger': bt.indicators.BollingerBands(d,
                    period=self.p.bollinger_period,
                    devfactor=self.p.devfactor),
                'atr': bt.indicators.ATR(d, period=self.p.atr_period),
                'adx': bt.indicators.ADX(d, period=self.p.adx_period),
                'vwap': bt.indicators.VWAP(d, period=self.p.vwap_period),
                'obv': bt.indicators.OBV(d),
                'obv_ema': bt.indicators.EMA(bt.indicators.OBV(d),
                    period=self.p.obv_ema),
                'volume_sma': bt.indicators.SMA(d.volume, period=10)
            }

    def next(self):
        for d in self.datas:
            pos = self.getposition(d).size
            indicators = self.indicators[d]
            
            # 趋势判断
            trend_up = (indicators['sma_fast'][0] > indicators['sma_slow'][0] and
                       indicators['macd'].macd[0] > indicators['macd'].signal[0] and
                       d.close[0] > indicators['bollinger'].lines.top[0])
            
            # 反转信号
            reversal = (indicators['rsi'][0] < 30 and
                       d.close[0] < indicators['bollinger'].lines.bot[0] and
                       indicators['obv'][0] > indicators['obv_ema'][0])
            
            # 动量确认
            momentum = (indicators['adx'][0] > 25 and
                       indicators['volume_sma'][0] > d.volume[-20:].mean())
            
            # 风险管理
            price = d.close[0]
            atr_stop = price - 2 * indicators['atr'][0]
            risk_amount = self.broker.getvalue() * self.p.risk_per_trade
            size = risk_amount / indicators['atr'][0]
            
            # 订单执行逻辑
            if not pos:
                if trend_up and momentum:
                    tp = price * (1 + self.p.takeprofit)
                    sl = atr_stop
                    main_order = self.buy(data=d, size=size)  # 开多仓主订单
    
                    # 止损单（市价单，触发后立即平仓）
                    stop_order = self.sell(data=d, 
                                        exectype=bt.Order.Stop,
                                        price=sl,
                                        parent=main_order,
                                        transmit=False)
                    
                    # 止盈单（限价单）
                    take_order = self.sell(data=d,
                                        exectype=bt.Order.Limit,
                                        price=tp,
                                        parent=main_order,
                                        transmit=False)
                    
                    # 设置OCO关系（任意一个成交则取消另一个）
                    stop_order.addinfo(take_order=take_order)
                    take_order.addinfo(stop_order=stop_order)
            else:
                if reversal or d.close[0] < sl:
                    self.close(data=d)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        if order.status in [order.Completed]:
            self.orders[order.data._name] = order
            if hasattr(order, 'take_order'):
                if order.take_order.status != order.Canceled: 
                    self.cancel(order.take_order)
            elif hasattr(order, 'stop_order'):
                if order.stop_order.status != order.Canceled:
                    self.cancel(order.stop_order)

In [10]:
def get_all_date_data(start_time, end_time, list_assets):
    data_path = 'etf_sina'

    # 从本地保存的数据中读出需要的股票日数据
    list_all = []
    for c in list_assets:
        df = pd.read_csv(f'{data_path}/{c}.csv')
        df['asset'] = c
        list_all.append(df[(df['date'] >= start_time) & (df['date'] <= end_time)])
        
    print(len(list_all))

    # 所有股票日数据拼接成一张表
    df_all = pd.concat(list_all) 
        
    # 返回计算因子需要的列
    df_all = df_all.reset_index()
    df_all = df_all[['date', 'asset', "open", "close", "high", "low", "volume", "pctChg"]]
    return df_all

In [3]:
fund_etf_top300_sina = pd.read_csv('fund_etf_top300_sina.csv')

In [5]:
list_assets = fund_etf_top300_sina['代码'].to_list()

In [17]:
start_time = '2000-01-01'
end_time = '2024-12-31'
fromdate = datetime.strptime(start_time, '%Y-%m-%d')
todate = datetime.strptime(end_time, '%Y-%m-%d')

In [ ]:
df_all = get_all_date_data(start_time, end_time, list_assets)

300


In [12]:
df1 = df_all.rename(columns={
        "date": "datetime", 
        "asset": "sec_code"})
df1["openinterest"] = 0
daily_price=df1[['sec_code','datetime', "open", "close", "high", "low", "volume", 'openinterest']]
daily_price['datetime'] = pd.to_datetime(daily_price['datetime'])
daily_price = daily_price.set_index(['datetime'])
daily_price

/tmp/ipykernel_7761/2658912843.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_price['datetime'] = pd.to_datetime(daily_price['datetime'])


,sec_code,open,close,high,low,volume,openinterest
datetime,,,,,,,
2013-04-09,sh511990,100.001,100.004,100.005,100.000,3222131,0
2013-04-10,sh511990,100.001,100.004,100.005,100.000,4732666,0
2013-04-11,sh511990,100.000,100.005,100.005,100.000,4003820,0
2013-04-12,sh511990,100.004,100.012,100.013,100.004,6152925,0
2013-04-15,sh511990,100.001,100.003,100.003,99.999,5230158,0
...,...,...,...,...,...,...,...
2024-12-25,sh588380,0.598,0.597,0.602,0.594,78228900,0
2024-12-26,sh588380,0.597,0.603,0.605,0.595,65402000,0
2024-12-27,sh588380,0.602,0.598,0.608,0.595,139486100,0


In [18]:

cerebro = bt.Cerebro()

for stock in daily_price['sec_code'].unique():
    # 日期对齐
    data = pd.DataFrame(index=daily_price.index.unique())
    df = daily_price.query(f"sec_code=='{stock}'")[['open','high','low','close','volume','openinterest']]
    data_ = pd.merge(data, df, left_index=True, right_index=True, how='left')
    data_.loc[:,['volume','openinterest']] = data_.loc[:,['volume','openinterest']].fillna(0)
    data_.loc[:,['open','high','low','close']] = data_.loc[:,['open','high','low','close']].fillna(method='pad')
    datafeed = bt.feeds.PandasData(dataname=data_, fromdate=fromdate, todate=todate)
    cerebro.adddata(datafeed, name=stock)
    print(f"{stock} Done !") 

sh511990 Done !
sh513330 Done !
sh511090 Done !
sh511880 Done !
sh511360 Done !
sh513130 Done !
sh513180 Done !
sh511520 Done !
sh513050 Done !
sh510300 Done !
sz159792 Done !
sh513080 Done !
sh512050 Done !
sh513030 Done !
sh513090 Done !
sh588000 Done !
sz159329 Done !
sh511380 Done !
sz159338 Done !
sz159361 Done !
sz159915 Done !
sh510050 Done !
sh563800 Done !
sh511260 Done !
sz159740 Done !
sh513060 Done !
sz159561 Done !
sh511130 Done !
sh512100 Done !
sh511010 Done !
sz159687 Done !
sh513770 Done !
sz159605 Done !
sh520830 Done !
sh512880 Done !
sh518880 Done !
sh588200 Done !
sh513120 Done !
sh511270 Done !
sz159568 Done !
sh510500 Done !
sz159920 Done !
sh512480 Done !
sz159529 Done !
sz159949 Done !
sh562500 Done !
sh510900 Done !
sh513980 Done !
sh510310 Done !
sh512000 Done !
sz159869 Done !
sz159509 Done !
sz159819 Done !
sh512170 Done !
sh513380 Done !
sz159816 Done !
sh513560 Done !
sz159636 Done !
sz159892 Done !
sh511220 Done !
sz159941 Done !
sz159919 Done !
sh512690

In [ ]:


cerebro.addstrategy(MultiStrategy)
cerebro.broker.setcash(1000000)
cerebro.broker.setcommission(commission=0.001)
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')
cerebro.addsizer(bt.sizers.PercentSizer, percents=90)

results = cerebro.run()
print(f'最终组合价值: {cerebro.broker.getvalue():.2f}')
cerebro.plot(style='candlestick')